In [237]:
import pandas as pd
import numpy as np
data=pd.read_csv('Sample_data.csv')
data['datetime'] = pd.to_datetime(data['start_ts'])
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['hour'] = data['datetime'].dt.hour
##week 1 +1，星期一是第一天
data['weekday'] = data['datetime'].dt.weekday+1
data['avg_speed'] = data['distance_km']/(data['duration_s']/3600)
#数据清洗
data = data.loc[(data['avg_speed']>0) &(data['avg_speed']<=100)]
data['avg_speed']=np.round(data['avg_speed'],decimals=-1)
print(data)
# data['trip_type'] = np.where((data['weekday']<=5 )& ((data['hour']>=6)  &(data['avg_speed']<=80)& (data['hour']<=9) | ((data['hour']>=17) & (data['hour']<=20))) ,'commute',
# (np.where((data['weekday']>=6) & (data['avg_speed']>=30),'weekend_trip','others')))
groupby = data.groupby(['weekday','avg_speed']).size().reset_index()
groupby.columns=['weekday','avg_speed','cnt']
sum_date = data.groupby(['weekday']).size().reset_index()
sum_date.columns=['weekday','cnt_by_weekday']
print(sum_date)
#add percentage
groupby = groupby.merge(sum_date,how='inner',on='weekday')
groupby['percentage'] = groupby['cnt']/groupby['cnt_by_weekday']
groupby['percentage']=np.round(groupby['percentage'],decimals=2)
groupby

        trip_id  distance_km  duration_s                 start_ts  \
0             0     9.000000      1323.0  2018-05-12 00:44:13.219   
1             1    10.000000      3720.0  2018-05-04 09:12:20.948   
2             2    36.900146      6841.0  2018-05-02 22:37:46.852   
3             3     8.900391      1441.0  2018-05-05 09:27:36.304   
4             4     6.200195      1081.0  2018-05-27 03:07:38.119   
...         ...          ...         ...                      ...   
299995   299995     7.199219      1202.0  2018-05-13 06:12:05.054   
299996   299996     0.400146       121.0  2018-05-21 05:19:51.085   
299997   299997     3.200195       601.0  2018-05-06 10:36:50.375   
299998   299998    19.200195      3843.0  2018-05-08 04:31:09.941   
299999   299999    10.099609      2521.0  2018-05-28 06:11:44.242   

                         end_ts  num_touchpad  num_speech  num_app_navi  \
0       2018-05-12 01:06:16.067           NaN         NaN           NaN   
1       2018-05-04 10

,weekday,avg_speed,cnt,cnt_by_weekday,percentage
0,1,0.0,2671,39706,0.07
1,1,10.0,11275,39706,0.28
2,1,20.0,12538,39706,0.32
3,1,30.0,6724,39706,0.17
4,1,40.0,3206,39706,0.08
...,...,...,...,...,...
72,7,60.0,743,35518,0.02
73,7,70.0,465,35518,0.01
74,7,80.0,338,35518,0.01
75,7,90.0,203,35518,0.01


In [238]:
# 看一下每日速度分布
import plotly.express as px
import plotly.graph_objects as go
fig = px.bar(groupby,x='weekday',y='cnt',color='avg_speed',text = 'percentage')
fig.show()

In [239]:

data['duration_s']=np.round(data['duration_s'],decimals=-3)
groupby = data.groupby(['weekday','duration_s']).size().reset_index()
groupby.columns=['weekday','duration_s','cnt']
print(groupby)

sum_date = data.groupby(['weekday']).size().reset_index()
sum_date.columns=['weekday','cnt_by_weekday']
print(sum_date)

#add percentage
groupby = groupby.merge(sum_date,how='inner',on='weekday')
groupby['percentage'] = groupby['cnt']/groupby['cnt_by_weekday']
groupby['percentage']=np.round(groupby['percentage'],decimals=2)
groupby
fig = px.bar(groupby,x='weekday',y='cnt',color='duration_s',text = 'percentage')
fig.show()

     weekday  duration_s    cnt
0          1         0.0  12197
1          1      1000.0  14544
2          1      2000.0   6038
3          1      3000.0   3142
4          1      4000.0   1436
..       ...         ...    ...
180        7     20000.0      1
181        7     21000.0      2
182        7     22000.0      1
183        7     23000.0      1
184        7     24000.0      2

[185 rows x 3 columns]
   weekday  cnt_by_weekday
0        1           39706
1        2           48433
2        3           49465
3        4           49621
4        5           39840
5        6           36638
6        7           35518


In [240]:
data['AM/PM'] = np.where((((data['hour']<=11))) ,'AM',
'PM')

groupby = data.groupby(['weekday','AM/PM']).size().reset_index()
groupby.columns=['weekday','AM/PM','cnt']
print(groupby)

sum_date = data.groupby(['weekday']).size().reset_index()
sum_date.columns=['weekday','cnt_by_weekday']
print(sum_date)

#add percentage
groupby = groupby.merge(sum_date,how='inner',on='weekday')
groupby['percentage'] = groupby['cnt']/groupby['cnt_by_weekday']
groupby['percentage']=np.round(groupby['percentage'],decimals=3)
groupby
fig = px.bar(groupby,x='weekday',y='cnt',color='AM/PM',text = 'percentage')
fig.show()

    weekday AM/PM    cnt
0         1    AM  31351
1         1    PM   8355
2         2    AM  37930
3         2    PM  10503
4         3    AM  39070
5         3    PM  10395
6         4    AM  38818
7         4    PM  10803
8         5    AM  31332
9         5    PM   8508
10        6    AM  28934
11        6    PM   7704
12        7    AM  27964
13        7    PM   7554
   weekday  cnt_by_weekday
0        1           39706
1        2           48433
2        3           49465
3        4           49621
4        5           39840
5        6           36638
6        7           35518


In [241]:
import plotly.graph_objects as go
num_touchpad = data.groupby("day")["num_touchpad"].sum()
num_speech = data.groupby("day")["num_speech"].sum()
num_app_navi = data.groupby("day")["num_app_navi"].sum()
num_app_player = data.groupby("day")["num_app_player"].sum()
num_guiding_start = data.groupby("day")["num_guiding_start"].sum()
num_app_radio = data.groupby("day")["num_app_radio"].sum()
merge = num_touchpad.to_frame().merge(num_speech,how='left',on='day').merge(num_app_navi,how='left',on='day').merge(num_app_player,how='left',on='day').merge(num_guiding_start,how='left',on='day').reset_index().merge(num_app_radio,how='left',on='day').reset_index()
print(merge)


fig_line = go.Figure()
fig_line.add_trace(go.Scatter(x=merge["day"],y=merge['num_touchpad'],
mode = 'lines',name='num_touchpad'))
fig_line.add_trace(go.Scatter(x=merge["day"],y=merge['num_app_player'],
mode = 'lines',name='num_app_player'))
fig_line.show()


fig_line2 = go.Figure()
fig_line2.add_trace(go.Scatter(x=merge["day"],y=merge['num_app_navi'],
mode = 'lines',name='num_app_navi'))
fig_line2.add_trace(go.Scatter(x=merge["day"],y=merge['num_app_radio'],
mode = 'lines',name='num_app_radio'))
fig_line2.show()


fig_line3 = go.Figure()
fig_line3.add_trace(go.Scatter(x=merge["day"],y=merge['num_speech'],
mode = 'lines',name='num_speech'))
fig_line3.add_trace(go.Scatter(x=merge["day"],y=merge['num_guiding_start'],
mode = 'lines',name='num_guiding_start'))
fig_line3.show()

# 没有明显结论

    index  day  num_touchpad  num_speech  num_app_navi  num_app_player  \
0       0    1       32399.0       176.0        4653.0         14037.0   
1       1    2       24051.0       187.0        3970.0         12040.0   
2       2    3       23461.0       198.0        4121.0         12429.0   
3       3    4       26565.0       171.0        4220.0         11997.0   
4       4    5       27775.0       138.0        3870.0         11706.0   
5       5    6       24153.0       148.0        4064.0         11783.0   
6       6    7       22923.0       196.0        4243.0         12188.0   
7       7    8       27993.0       277.0        4249.0         11915.0   
8       8    9       23491.0       198.0        4257.0         11639.0   
9       9   10       25354.0       180.0        4203.0         11869.0   
10     10   11       25399.0       189.0        4463.0         12772.0   
11     11   12       23882.0       189.0        4064.0         12287.0   
12     12   13       25632.0       168

In [271]:
# 给旅程分类
data['weekend'] = np.where((data['weekday']<=5 )& (data['AM/PM']=='AM') ,'workday_AM',
(np.where((data['weekday']<=5) & (data['AM/PM']=='PM'),'workday_PM',(np.where((data['weekday']>5) & (data['AM/PM']=='AM'),'weenend_AM',(np.where((data['weekday']>5) & (data['AM/PM']=='PM'),'weekend_PM','others')))))))
groupby_weekend = data.groupby("weekend")["trip_id"].count().reset_index()
groupby_weekend['percentage'] = groupby_weekend['trip_id']/groupby_weekend['trip_id'].sum()*100

data['duration_s']=np.round(data['duration_s'],decimals=3)
groupby_navi = data.groupby("weekend")["num_touchpad"].sum().reset_index()
groupby_speech = data.groupby("weekend")["num_speech"].sum().reset_index()
groupby_num_app_navi=data.groupby("weekend")["num_app_navi"].sum().reset_index()
groupby_num_app_player =data.groupby("weekend")["num_app_player"].sum().reset_index()
groupby_num_app_radio =data.groupby("weekend")["num_app_radio"].sum().reset_index()
groupby_num_guiding_start =data.groupby("weekend")["num_guiding_start"].sum().reset_index()

groupby_num_distance =data.groupby("weekend")["distance_km"].mean().reset_index()
groupby_num_speed =data.groupby("weekend")["avg_speed"].mean().reset_index()
groupby_num_durations =data.groupby("weekend")["duration_s"].mean().reset_index()
print(groupby_num_speed)


print(groupby_navi)

print(groupby_weekend)

df = groupby_weekend.merge(groupby_navi,how='left',on='weekend').merge(groupby_speech,how='left',on='weekend').merge(groupby_num_app_navi,how='left',on='weekend') \
.merge(groupby_num_app_player,how='left',on='weekend').merge(groupby_num_app_radio,how='left',on='weekend').merge(groupby_num_guiding_start,how='left',on='weekend') \
    .merge(groupby_num_distance,how='left',on='weekend').merge(groupby_num_speed,how='left',on='weekend').merge(groupby_num_durations,how='left',on='weekend')

df['touchpad_rate_per_100km'] = df['num_touchpad']/df['distance_km']*100
df['num_speech_per_100km'] = df['num_speech']/df['distance_km']*100
df['num_app_navi_per_100km'] = df['num_app_navi']/df['distance_km']*100
df['num_app_player_per_100km'] = df['num_app_player']/df['distance_km']*100
df['num_app_radio_per_100km'] = df['num_app_radio']/df['distance_km']*100
df['num_guiding_start_per_100km'] = df['num_guiding_start']/df['distance_km']*100
print(df)


fig_line4 = go.Figure()
fig_line4.add_trace(go.Scatter(x=df["weekend"],y=df['touchpad_rate_per_100km'],
mode = 'lines',name='touchpad_rate_per_100km'))
fig_line4.add_trace(go.Scatter(x=df["weekend"],y=df['num_app_player_per_100km'],
mode = 'lines',name='num_app_player_per_100km'))
fig_line4.show()

fig_line5 = go.Figure()
fig_line5.add_trace(go.Scatter(x=df["weekend"],y=df['num_guiding_start_per_100km'],
mode = 'lines',name='num_guiding_start_per_100km'))
fig_line5.add_trace(go.Scatter(x=df["weekend"],y=df['num_speech_per_100km'],
mode = 'lines',name='num_speech_per_100km'))
fig_line5.show()


fig_line6 = go.Figure()
fig_line6.add_trace(go.Scatter(x=df["weekend"],y=df['num_app_radio_per_100km'],
mode = 'lines',name='num_app_radio_per_100km'))
fig_line6.add_trace(go.Scatter(x=df["weekend"],y=df['num_app_navi_per_100km'],
mode = 'lines',name='num_app_navi_per_100km'))
fig_line6.show()


fig_line7 = go.Figure()
fig_line7.add_trace(go.Scatter(x=df["weekend"],y=df['avg_speed'],
mode = 'lines',name='avg_speed'))
fig_line7.add_trace(go.Scatter(x=df["weekend"],y=df['distance_km'],
mode = 'lines',name='distance_km'))
fig_line7.show()


      weekend  avg_speed
0  weekend_PM  24.138812
1  weenend_AM  22.584449
2  workday_AM  22.395729
3  workday_PM  23.740219
      weekend  num_touchpad
0  weekend_PM       43895.0
1  weenend_AM      158054.0
2  workday_AM      452609.0
3  workday_PM      129536.0
      weekend  trip_id  percentage
0  weekend_PM    15258    5.099241
1  weenend_AM    56898   19.015377
2  workday_AM   178501   59.655238
3  workday_PM    48564   16.230144
      weekend  trip_id  percentage  num_touchpad  num_speech  num_app_navi  \
0  weekend_PM    15258    5.099241       43895.0       188.0        6647.0   
1  weenend_AM    56898   19.015377      158054.0      1144.0       26181.0   
2  workday_AM   178501   59.655238      452609.0      3388.0       77766.0   
3  workday_PM    48564   16.230144      129536.0       709.0       19674.0   

   num_app_player  num_app_radio  num_guiding_start  distance_km  avg_speed  \
0         19967.0         9621.0              232.0    14.409032  24.138812   
1         7